In [1]:
# import libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import MapboxTiles
import geopandas as gpd
import os
from dotenv import load_dotenv
load_dotenv() # loads .env file from main folder for Mapbox API key and local project path
import random
import numpy as np

/opt/homebrew/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.12.0-CAPI-1.18.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# load parcel and building data
df_parcels = gpd.read_file(os.environ.get('LOCAL_PATH')+"Data/Parcels/NYC_2021_Tax_Parcels_SHP_2203/NewYork_2021_Tax_Parcels_SHP_2203.shp")
df_buildings = gpd.read_file(os.environ.get('LOCAL_PATH')+"Data/Building Footprints/NYC/geo_export_a80ea1a2-e8e0-4ffd-862c-1199433ac303.shp")

In [4]:
# set random RGB color for the parcels
def random_hex_color(seed=False):
  if seed:
    random.seed(seed)
    r = random.randint(0, 255)
    random.seed(seed+1000)
    g = random.randint(0, 255)
    random.seed(seed+2000)
    b = random.randint(0, 255)
  else:
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
  return "#{:02x}{:02x}{:02x}".format(r, g, b)
random_hex_color()
df_parcels['color'] = [ random_hex_color() for i in range(len(df_parcels)) ]

In [4]:
# join dataframes function
def join_parcels_buildings(parcels, buildings):
    """Join parcels and buildings dataframes."""
    parcels_buildings = buildings.sjoin(parcels, how="inner")
    return parcels_buildings

In [5]:
# set CRS to web mercator
df_parcels = df_parcels.to_crs(epsg=3857)
df_buildings = df_buildings.to_crs(epsg=3857)

In [6]:
# perform join
df_parcels_buildings = join_parcels_buildings(df_parcels, df_buildings)

In [ ]:
# show df length
print(df_parcels.shape[0])
print(df_buildings.shape[0])
print(df_parcels_buildings.shape[0])

In [7]:
# loop through the parcels and add them to the map
def add_geometries(ax, df_parcels, crs_epsg, random_color = False):
    for row in df_parcels.itertuples():
        geometry = row.geometry
        if random_color == True: color = random_hex_color(int(row.bin))
        else: color = row.color
        ax.add_geometries(geometry, crs = crs_epsg, facecolor=color) # for Lat/Lon data.

In [8]:
# load the background map and plot the geometries
# def map_maker(df_parcels, df_buildings, scale=10):
def map_maker(df_parcels, df_buildings, bounds, index, scale=10, feature_type='both', random_color=False):
    access_token = os.environ.get('MAPBOX_ACCESS_TOKEN')
    tiler = MapboxTiles(access_token, 'satellite-v9')
    crs_epsg = ccrs.epsg('3857')

    mercator = tiler.crs

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection=mercator)

    # change figure size of the subplot
    my_dpi=96
    fig.set_size_inches(7, 7)
    # fig.figsize = (512/my_dpi, 512/my_dpi), dpi=my_dpi

    # calculate the centroid and max distance of the bounds
    dist1 = bounds[2]-bounds[0]
    dist2 = bounds[3]-bounds[1]
    max_dist = max(dist1, dist2)/2

    # calculate the centroid of the bounds
    centroid_x = (bounds[2]+bounds[0])/2
    centroid_y = (bounds[3]+bounds[1])/2

    # bounds = df_parcels.total_bounds with offset to create same aspect ratio
    ax.set_extent([centroid_x-max_dist, centroid_x+max_dist, centroid_y-max_dist, centroid_y+max_dist], crs=ccrs.epsg('3857'))

    # if feature_type == 'parcels': add_parcels(ax, df_parcels, crs_epsg)
    if feature_type == 'parcels': 
        add_geometries(ax, df_parcels, crs_epsg)
        # ax.add_geometries(df_buildings.geometry, crs = crs_epsg, facecolor='white', edgecolor='black', linewidth=2.5, alpha=1)
    if feature_type == 'parcels' and random_color == True:
        add_geometries(ax, df_parcels, crs_epsg, random_color=True)
    if feature_type == 'buildings': 
        add_geometries(ax, df_buildings, crs_epsg)
    if feature_type == 'buildings' and random_color == True:
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
    if feature_type == 'both' and random_color == True: 
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
        add_geometries(ax, df_parcels, crs_epsg, random_color=True)
    if  feature_type == 'both': 
        # ax.add_geometries(df_buildings.geometry, crs = crs_epsg, facecolor='black', edgecolor='white', linewidth=1.5, alpha=1)
        add_geometries(ax, df_buildings, crs_epsg, random_color=True)
        add_geometries(ax, df_parcels, crs_epsg)

    # add the Mapbox tiles to the axis at zoom level 10 (Mapbox has 23 zoom levels)
    ax.add_image(tiler, scale)

    # set the path to the folder where the images will be saved
    output_folder = 'img/'

    # save the figure
    plt.savefig(output_folder + f'{feature_type}_{index}.jpg', bbox_inches='tight', pad_inches = 0, dpi = my_dpi)
    # plt.savefig(output_folder + f'building_{index}.jpg', bbox_inches='tight', pad_inches = 0, dpi = my_dpi)

    # close the figure
    plt.close(fig)

    # ax.coastlines('10m')
    plt.show()

In [9]:
# subset the data frames based on a buffer
def subset(df, df_buildings, index, distance = 75):
    selected_feature = df.loc[index]
    geometry_buffer = selected_feature.geometry.buffer(distance)
    geometry_bounds = selected_feature.geometry.buffer(distance-70)

    return df[df.within(geometry_buffer)], df_buildings[df_buildings.within(geometry_buffer)], geometry_bounds.bounds

In [ ]:
# create a folder to store the images if it doesn't exist
# make sure to git-ignore the folder
os.mkdir('img')

In [11]:
# generate the image for "parcels" and "buildings" (or "both")
# index will correspond to each of the parcel indices in the data frame
for i in range (0, 24999):
    subset_features = subset(df_parcels, df_parcels_buildings, i, 200)
    map_maker(subset_features[0], subset_features[1], subset_features[2], i, 18,'parcels')
    # map_maker(subset_features[0], subset_features[1], subset_features[2], i, 18,'buildings')